In [195]:
import pandas as pd
import numpy as np

In [196]:
bank_df = pd.read_excel("bank_deposit.xlsx")
bank_df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,Bank deposit(target)
0,95,retired,divorced,primary,no,2282.0,no,no,telephone,21.0,apr,207.0,17.0,-1.0,0.0,unknown,yes
1,95,retired,married,secondary,no,0.0,no,no,telephone,1.0,oct,215.0,1.0,-1.0,0.0,unknown,no
2,94,retired,divorced,secondary,no,1234.0,no,no,cellular,3.0,mar,212.0,1.0,-1.0,0.0,unknown,no
3,93,retired,married,unknown,no,775.0,no,no,cellular,22.0,jul,860.0,2.0,177.0,7.0,success,yes
4,93,retired,married,unknown,no,775.0,no,no,cellular,4.0,aug,476.0,2.0,13.0,9.0,success,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11217,50,entrepreneur,divorced,tertiary,no,315.0,no,no,cellular,12.0,jul,101.0,2.0,185.0,2.0,failure,no
11218,50,management,married,tertiary,no,2881.0,no,no,cellular,5.0,aug,510.0,2.0,2.0,5.0,other,no
11219,50,technician,divorced,secondary,no,0.0,yes,no,cellular,13.0,aug,120.0,2.0,91.0,6.0,failure,no
11220,50,services,married,secondary,no,2376.0,yes,no,cellular,24.0,sep,395.0,1.0,-1.0,0.0,unknown,yes


In [ ]:
# Dropped the rows that contained NAN in the contact column.
bank_df= bank_df[bank_df['contact'].notna()]
bank_df

In [ ]:
bank_df.crosstab('Bank deposit(target)','count')

In [ ]:
def quantile_25(bank_df:pd.Series):
    return bank_df.quantile(.25)

def quantile_75(bank_df:pd.Series):
    return bank_df.quantile(.75)

In [ ]:
bank_pivot=bank_df.pivot_table(values='balance', index=['Bank deposit(target)','loan'], aggfunc=[ quantile_25, np.median, quantile_75,np.max])
bank_pivot

quantile_25  median quantile_75        amax
                              balance balance     balance     balance
Bank deposit(target) loan                                            
no                   no        111.00   638.0     2007.00  68000000.0
                     yes        16.25   313.0     1137.25     51439.0
yes                  no        346.00  1195.5     2998.25     52587.0
                     yes       104.00   437.5     1745.75      9103.0

In [264]:
# Droping customer index 38 because it is a potential duplicate with corrupted "duration" value.
bank_df = bank_df.drop(index=38)

In [243]:
#Median1 is the value for Bank deposit=no, loan=no
median1=int(bank_pivot.loc[(["no"],["no"]),(["median"],["balance"])].values[0])
#Median2 is the value for Bank deposit=yes, loan=no
median2=int(bank_pivot.loc[(["yes"],["no"]),(["median"],["balance"])].values[0])

#These filters allows us to check for multiple conditions and change the "loan" column according to these conditions with a .loc method afterwards.
filter1 = (bank_df['loan'].isna()) & (bank_df['Bank deposit(target)']=="no") & (bank_df['balance'] > median1)
bank_df.loc[filter1, "loan"] = "no"
filter2 = (bank_df['loan'].isna()) & (bank_df['Bank deposit(target)']=="no") & (bank_df['balance'] < median1)
bank_df.loc[filter2, "loan"] = "yes"
filter3 = (bank_df['loan'].isna()) & (bank_df['Bank deposit(target)']=="yes") & (bank_df['balance'] > median2)
bank_df.loc[filter3, "loan"] = "no"
filter4 = (bank_df['loan'].isna()) & (bank_df['Bank deposit(target)']=="yes") & (bank_df['balance'] < median2)
bank_df.loc[filter4, "loan"] = "yes"

In [ ]:
#Changed NAN into unknown in the "education" column.
bank_df["education"] = bank_df["education"].fillna("unknown")

In [ ]:
#This filter checks if Bank deposit(target) is yes and changes the poutcome to success if the condition is met.
filter_bank_deposit_yes = bank_df["Bank deposit(target)"] == "yes"
bank_df.loc[filter_bank_deposit_yes, "poutcome"] = "success"